# Infosys Quarterly Report Analysis

#### Developed By: Manaranjan Pradhan
#### www.manaranjanp.com

*This Jupyter notebook is confidential and proprietary to Manaranjan Pradhan. It is intended solely for authorized training purposes. Unauthorized distribution, sharing, or reproduction of this notebook or its contents is strictly prohibited. This material is for personal learning within the training program only and may not be used for commercial purposes or shared with others. Unauthorized use may result in disciplinary action or legal consequences. If you have received this notebook without authorization, please contact manaranjan@gmail.com immediately and delete all copies.*

In [ ]:
!pip install getpass4 pypdf openai faiss-cpu llama-index llama-index-readers-file llama-index-vector-stores-faiss llama-index-embeddings-openai llama-index-llms-groq llama-index-embeddings-huggingface

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 11.8 MB/s eta 0:00:00


In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import os
from getpass import getpass
os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [ ]:
from llama_index.core import SimpleDirectoryReader, ServiceContext, VectorStoreIndex, StorageContext
from llama_index.core.response.pprint_utils import pprint_response
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.node_parser import (SentenceWindowNodeParser,)
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.core import Document
import faiss
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [ ]:
import faiss

## Configure LLM service

In [ ]:
#llm = OpenAI(temperature=0,
#             model="chatgpt-4o-latest",
#             max_tokens=500)


llm = Groq(model="llama-3.3-70b-versatile")

In [ ]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 3900

## Load data
Downloaded from

https://www.infosys.com/investors/reports-filings/quarterly-results.html

In [ ]:
!pip install docling

In [ ]:
from docling.document_converter import DocumentConverter

In [ ]:
converter = DocumentConverter()
result = converter.convert("/content/ifrs-inr-press-release.pdf")
result.document.export_to_markdown()

# Define the output directory
output_dir = "output_documents"
os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists

# Define output file path
output_path = os.path.join(output_dir, "converted_document.md")

# Save as Markdown
with open(output_path, "w", encoding="utf-8") as f:
    f.write(result.document.export_to_markdown())

print(f"Document saved at: {output_path}")


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Document saved at: output_documents/converted_document.md


In [ ]:
#q1_2024 = SimpleDirectoryReader(
#    input_files=["/content/ifrs-inr-press-release.pdf"]
#).load_data()

q1_2024 = SimpleDirectoryReader(
    input_files=["/content/output_documents/converted_document.md"]
).load_data()

# Build indices

In [ ]:
# dimensions of text-ada-embedding-002
d = 384
faiss_index = faiss.IndexFlatL2(d)

In [ ]:
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
q1_2024_index = VectorStoreIndex.from_documents(q1_2024, storage_context=storage_context)

## Build query engines

In [ ]:
q1_2024__engine = q1_2024_index.as_query_engine(similarity_top_k=2)

In [ ]:
# Assuming you have already built the index and query engine
retriever = q1_2024__engine.retriever
retrieved_nodes = retriever.retrieve("What is the revenue growth for the quarter?")

# Print the retrieved chunks
for i, node in enumerate(retrieved_nodes):
    print(f"\n--- Chunk {i+1} ---")
    print(node.get_text())


--- Chunk 1 ---
945                          | 5,360                          |
| Basic EPS ( ` )                       | 14.37                          | 12.78                          |
| Diluted EPS ( ` )                     | 14.35                          | 12.76                          |

<!-- image -->

## NOTES:

1. The above information is extracted from the audited condensed consolidated Balance sheet and Statement of Comprehensive Income for the quarter ended June 30, 2023, which have been taken on record at the Board meeting held on July 20, 2023.
2. A Fact Sheet providing the operating metrics of the Company can be downloaded from www.infosys.com.
3. Represents bank balance earmarked for final dividend. Payment date for dividend was July 3, 2023.
4. Other income is net of Finance Cost.

--- Chunk 2 ---
<!-- image -->

\

Solid Q1 year on year revenue growth of 4.2% at 20.8% operating margins Strong large deal closures and robust deal pipeline position us well for future 

## Run queries

In [ ]:
response = q1_2024__engine.query(
    "What is the revenue growth for the quarter?"
)

In [ ]:
q1_2024__engine.get_prompts()

{'response_synthesizer:text_qa_template': SelectorPromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings={}, function_mappings={}, default_template=PromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: '), conditionals=[(<function is_chat_model at 0x7cc6733f2480>, ChatPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, message_templates=[ChatMessage(role=<MessageRole.

In [ ]:
print(response)

4.2% year on year revenue growth and 1.0% sequential growth in constant currency.


In [ ]:
response = q1_2024__engine.query("Which are some of the key customer wins?")

In [ ]:
print(response)

Some of the key customer wins include XacBank, where Infosys Finacle helped transform their technology landscape with the Finacle Digital Banking Suite, enabling a robust digital foundation for the bank to achieve its growth strategy.


In [ ]:
response = q1_2024__engine.query("What are some of the key achievements in the quarter?")

In [ ]:
print(response)

Some of the key achievements in the quarter include revenues growing by 4.2% YoY and 1.0% QoQ in CC terms, reported revenues reaching ` 37,933 crore with a 10.0% YoY growth, operating margin at 20.8%, Basic EPS at ` 14.37 with a 12.4% YoY growth, and FCF at ` 5,749 crore with a 12.6% YoY growth and a FCF conversion of 96.7% of net profit.


In [ ]:
response = q1_2024__engine.query("What is the total assets?")

In [ ]:
print(response)

131,322
